In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.datasets import cifar10
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.models import Model


In [ ]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert labels to 1D arrays
y_train = y_train.flatten()
y_test = y_test.flatten()

# Print shape of the dataset
print(f"Training data shape: {x_train.shape}")
print(f"Test data shape: {x_test.shape}")


In [ ]:
# Load the ResNet50 model pre-trained on ImageNet
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(32, 32, 3))

# Create a new model to extract features from the last layer
feature_extractor = Model(inputs=resnet.input, outputs=resnet.layers[-1].output)

# Preprocess the images and extract features
def extract_features(images):
    images = preprocess_input(images)  # Preprocess input for ResNet50
    features = feature_extractor.predict(images)
    return features.reshape(features.shape[0], -1)  # Flatten the features

# Extract features from the training and test data
x_train_features = extract_features(x_train)
x_test_features = extract_features(x_test)

# Print the shape of the extracted features
print(f"Training features shape: {x_train_features.shape}")
print(f"Test features shape: {x_test_features.shape}")


In [ ]:
# Train an SVM classifier using the extracted features
svm_model = SVC(kernel='rbf', C=1, gamma='auto')
svm_model.fit(x_train_features, y_train)

# Predict on the test set
y_pred = svm_model.predict(x_test_features)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
plt.imshow(conf_matrix, cmap="Blues", interpolation="none")
plt.title("Confusion Matrix")
plt.colorbar()
plt.show()


In [ ]:
# Helper function to plot a few test images with their predicted and true labels
def plot_predictions(images, true_labels, pred_labels, class_names, n=10):
    plt.figure(figsize=(15, 5))
    for i in range(n):
        plt.subplot(2, 5, i + 1)
        plt.imshow(images[i])
        plt.title(f"True: {class_names[true_labels[i]]}\nPred: {class_names[pred_labels[i]]}")
        plt.axis('off')

# CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Plot first 10 test images with predictions
plot_predictions(x_test[:10], y_test[:10], y_pred[:10], class_names)
